In [1]:
import pandas as pd
import sqlalchemy as sql
import json
import numpy as np

In [2]:
from sql_connection import engine
engine.connect()

In [3]:
query_recptime = """
select 
    recpid,
    recpname,
    recpversion,
    trackintime,
    startrecptime,
    endrecptime,
    trackouttime, 
    case
        when endrecptime is not null and startrecptime is not null then trunc((endrecptime - startrecptime)*86400)
        when trackouttime is not null and trackintime is not null then trunc((trackouttime - trackintime)*86400)
        else 0
    end as recptime
from plld.hist@sgodsprd
where 
    prodarea in ('SINGAPORE','SINGAPORE3')
    and lottype in ('E', 'P')
    and TIMESTAMPTIME BETWEEN TO_TIMESTAMP('01-JAN-23 00.00.00', 'DD-MON-YY HH24.MI.SS') and TO_TIMESTAMP('31-DEC-24 23.59.59', 'DD-MON-YY HH24.MI.SS')
"""

df_recptime = pd.read_sql(query_recptime, con=engine)
print(df_recptime.head())

     recpid recpname recpversion         trackintime       startrecptime  \
0  SWT02.01    SWT02         .01 2023-01-01 10:36:47 2023-01-01 10:36:47   
1  SOW04.01    SOW04         .01 2023-01-01 09:12:53 2023-01-01 09:12:53   
2  SOK03.06    SOK03         .06 2023-01-01 10:26:08 2023-01-01 10:26:08   
3  3BGC1.01    3BGC1         .01 2023-01-01 09:13:10 2023-01-01 09:13:10   
4  3CMA1.06    3CMA1         .06 2023-01-01 10:37:10 2023-01-01 10:37:10   

          endrecptime        trackouttime  recptime  
0 2023-01-01 10:36:49 2023-01-01 10:36:49         2  
1 2023-01-01 10:36:59 2023-01-01 10:36:59      5046  
2 2023-01-01 10:37:09 2023-01-01 10:37:09       661  
3 2023-01-01 10:37:10 2023-01-01 10:37:10      5039  
4 2023-01-01 10:37:10 2023-01-01 10:37:10         0  


In [4]:
# drop null values
df_recptime = df_recptime.replace(' ', pd.NA).dropna(subset=['recpid', 'recpname', 'recpversion'])
# either all not null or trackin and trackout not null & startrecptime and endrecptime null
df_recptime = df_recptime[(df_recptime[['trackintime', 'startrecptime', 'endrecptime', 'trackouttime']].notnull().all(axis=1))
                           | (df_recptime[['trackintime', 'trackouttime']].notnull().all(axis=1) & df_recptime[['startrecptime', 'endrecptime']].isnull().all(axis=1))]


In [9]:
def process_group(group):
    group = group.sort_values('recptime')
   
    # remove top and tail 5% of times
    n = len(group)
    if n < 10:
        trimmed_times = group[:]
    else:
        trimmed_times = group[int(0.05 * n + 1): int(0.95 * n)]
    count = len(trimmed_times)
    # Calculation
    if count <=1:
        upper = np.max(trimmed_times['recptime'])
        lower = np.min(trimmed_times['recptime'])
        result = {
        'recpid': group['recpid'].iloc[0],
        'recpname': group['recpname'].iloc[0],
        'recpversion': group['recpversion'].iloc[0],  
        'count': count,
        'mean': int(np.mean(trimmed_times['recptime'])),
        # 'mode': trimmed_times['recptime'].mode()[0] if len(trimmed_times['recptime'].mode()) > 0 else None,
        # 'SD': 0,
        # 'lower_cutoff': lower,
        # 'upper_cutoff': upper,
        # 'mean - 2SD': lower,
        # 'mean + 2SD': upper,
        'lower_bound': lower,
        'upper_bound': upper
    }
    else:
        mean = int(np.mean(trimmed_times['recptime']))
        std = np.std(trimmed_times['recptime'])
        # mode = trimmed_times['recptime'].mode()

        lower_cutoff = np.min(trimmed_times['recptime'])
        upper_cutoff = np.max(trimmed_times['recptime'])
        # take less extreme of 2SD and min/max
        lower_bound = int(max(mean - 2 * std, lower_cutoff))
        upper_bound = int(min(mean + 2 * std, upper_cutoff))

        result = {
            'recpid': group['recpid'].iloc[0],
            'recpname': group['recpname'].iloc[0],
            'recpversion': group['recpversion'].iloc[0],  
            'count': count,
            'mean': mean,
            # 'mode': mode[0],    
            # 'SD': std,
            # 'lower_cutoff': lower_cutoff,
            # 'upper_cutoff': upper_cutoff,
            # 'mean - 2SD': mean - 2 * std,
            # 'mean + 2SD': mean + 2 * std,
            'lower_bound': lower_bound,
            'upper_bound': upper_bound
        }
    return result

recptime_all_version = df_recptime.groupby('recpid').apply(process_group)

df_recptime_all_version = pd.DataFrame(recptime_all_version.tolist())  

df_recptime_all_version.head()


C:\Users\reny2\AppData\Local\Temp\ipykernel_4316\4058484000.py:58: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  recptime_all_version = df_recptime.groupby('recpid').apply(process_group)


,recpid,recpname,recpversion,count,mean,lower_bound,upper_bound
0,33RF1.03,33RF1,.03,5,32,14,88
1,36RF1.03,36RF1,.03,6,140,14,295
2,3AGC1.04,3AGC1,.04,5051,1469,602,2786
3,3AGC1.05,3AGC1,.05,291,1685,123,3246
4,3AGC1.06,3AGC1,.06,734,1864,665,3506


In [6]:
# # check for recpids with only 1 record; 25 ids
# df_recptime_all_version[df_recptime_all_version['count'] <=1]

In [10]:
# export to csv
df_recptime_all_version.to_csv('recptime_all_version.csv', index=False)

In [11]:
query_recp = """
select 
    recpname,
    recpversion,
    activeflag
from plld.recp@sgodsprd
"""

df_recp = pd.read_sql(query_recp, con=engine)
print(df_recp.head())

  recpname recpversion activeflag
0    33RF1         .01           
1    36RF1         .01           
2    3AGC1         .01           
3    3AGC1         .02           
4    3AGC1         .03           


In [12]:
def get_active_recp(df_recp, recp_name):
    """Fetch the active row (ActiveFlag = 'A') from df_prcd based on prcd_name."""
    active_row = df_recp[(df_recp['recpname'] == recp_name) & (df_recp['activeflag'] == 'A')]
    return active_row.iloc[0]['recpversion'] if not active_row.empty else None

In [13]:
def fetch_recp_time(df_recp_time, recp_name, recp_version):
    """use the correct name and version to identify the range of recp time and generate a random recp time"""
    results = df_recp_time[(df_recp_time['recpname'] == recp_name) & (df_recp_time['recpversion'] == recp_version)]
    # if there is no data, use previous version's data
    counter = 0
    while results.empty and counter < 50:
        recp_version = "{:.2f}".format(float(recp_version) - 0.01).lstrip('0')
        # print(recp_version)
        results = df_recp_time[(df_recp_time['recpname'] == recp_name) & (df_recp_time['recpversion'] == recp_version)]
        counter += 1
    if results.empty:
        return None, None, None
    mean = results.iloc[0]['mean']
    lower_bound = results.iloc[0]['lower_bound']
    upper_bound = results.iloc[0]['upper_bound']
    return mean, lower_bound, upper_bound
    # return np.random.uniform(lower_bound, upper_bound, SEED)

In [14]:
# get df_recptime_active_version
recptime_active_version = {}
for name in df_recptime_all_version['recpname'].unique():
    active_version = get_active_recp(df_recp, name)
    if active_version is None:
        recptime_active_version[name] = {
        "recpname": name,
        "recpversion": None,
        "mean":None,
        "active_lower_bound": None,
        "active_upper_bound": None
    }
    mean, active_lower, active_upper = fetch_recp_time(df_recptime_all_version, name, active_version)
    recptime_active_version[name] = {
        "recpname": name,
        "recpversion": active_version, # note that this is always the latest version, might not be the version where the data is from
        "mean": mean,
        "active_lower_bound": active_lower,
        "active_upper_bound": active_upper
    }
# make it a dataframe
df_recptime_active_version = pd.DataFrame(recptime_active_version.values())
df_recptime_active_version.head()


,recpname,recpversion,mean,active_lower_bound,active_upper_bound
0,33RF1,.03,32,14,88
1,36RF1,.03,140,14,295
2,3AGC1,.07,1688,651,3497
3,3AH01,.02,0,0,0
4,3AOI1,.02,1492,2,4557


In [53]:
# # locate recpids with none in activw_lower_bound and active_upper_bound
# df_recptime_active_version[(df_recptime_active_version['active_lower_bound'].isnull()) & (df_recptime_active_version['active_upper_bound'].isnull())]

In [15]:
# export to csv
df_recptime_active_version.to_csv('recptime_active_version.csv', index=False)